In this example, we relax the phase retrieval problem similar to the classical [MaxCut](http://www-math.mit.edu/~goemans/PAPERS/maxcut-jacm.pdf) semidefinite program and recover the phase of the signal given the magnitude of the linear measurements.

Phase recovery has wide applications such as  in X-ray and crystallography imaging, diffraction imaging or microscopy and audio signal processing. In all these applications, the detectors cannot measure the phase of the incoming wave and only record its amplitude i.e complex measurements of a signal $x \in \mathbb{C}^p$ are obtained from a linear injective operator A, **but we can only measure the magnitude vector |Ax| not the phase fo Ax**.

Recovering the phase of Ax from |Ax| is a **nonconvex optimization problem**. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

The original reprsentation of the problem is as follows:

>>>> find x

>>>> such that |Ax| = b

>>>> where $x \in \mathbb{C}^p$, $A \in \mathbb{C}^{n×p}$ and $b \in \mathbb{R}^n$.



In this example, **the problem is to find the phase of Ax given the value |Ax|**. Given a linear operator $A$ and a vector $b= |Ax|$ of measured amplitudes, in the noiseless case, we can write Ax = diag(b)u where $u \in \mathbb{C}^n$ is a phase vector, satisfying |$\mathbb{u}_i$| = 1 for i = 1,. . . , n. 

We relax this problem as Complex Semidefinite Programming.

### Relaxed Problem similar to [MaxCut](http://www-math.mit.edu/~goemans/PAPERS/maxcut-jacm.pdf)

Define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.

In [13]:
using Convex

n = 4
p = 2
A = rand(n,p) + im*randn(n,p)
x = rand(p) + im*randn(p)
b = abs(A*x) + rand(n)

M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((8,8),(8,8))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 125
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 33, constraints m = 73
Cones:	primal zero / dual free vars: 37
	sd vars: 36, sd blks: 1
Setup time: 9.04e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  5.01e-03 
   100| 9.73e-03  1.35e+00  1.16e-03 -9.00e+01 -8.98e+01  1.18e-15  1.41e-02 
   200| 9.73e-05  2.49e-02  4.17e-06 -9.03e+01 -9.03e+01  1.36e-15  2.48e-02 
   300| 

4×4 Array{Complex{Float64},2}:
 1.0-1.3529e-17im            0.854607-0.519275im  …      0.999839+0.017965im
    0.854607+0.519275im  1.0+2.58732e-17im                0.84514+0.534545im
    0.951344-0.30813im       0.653021-0.75734im          0.956726-0.29099im 
    0.999839-0.017965im       0.84514-0.534545im     1.0+4.09013e-17im      

In [14]:
# Verify if the rank of U is 1
B, C = eig(U.value);
print(length([e for e in B if(abs(real(e))>1e-4)]))
#Decompose U = uu*
# u is the phase of Ax
u = C[:,1]

1

4-element Array{Complex{Float64},1}:
 0.475672+0.154065im
  0.32651+0.37867im 
      0.5+0.0im     
 0.478363+0.145495im